# Importing the necessary libraries

In [1]:
import os
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pyttsx3
import pyautogui
import os
import time
import pandas as pd

# Loading the saved model

In [2]:
new_model = tf.keras.models.load_model('fer_model_25epochs.keras')

# REALTIME VIDEO DEMO

In [3]:
import cv2
path = 'haarcascade_frontalface_default.xml'
font = cv2.FONT_HERSHEY_TRIPLEX
font_scale = 2
font_thickness = 2
font_colour = (0, 0, 225)

In [4]:
def display_text(emotion):
    # Rectangular bounding box for the face
    cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255))
    # Adding text
    cv2.putText(frame, emotion, (x, y-10), font, font_scale, font_colour, font_thickness, cv2.LINE_4)

In [17]:
cap = cv2.VideoCapture(1)

# Checking if the webcam is opening
if not cap.isOpened():
    cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise IOError('Canot open webcam!')

while True:
    
    ret, frame = cap.read()
    faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray, 1.1, 4)
    
    for x, y, w, h in faces:
    
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        cv2.rectangle(frame, (x,y), (x+w, y+h), (255, 0, 0), 2)

        facess = faceCascade.detectMultiScale(roi_gray)

        if len(facess) == 0:
            print('Face not detected!')
        else:
            for (ex, ey, ew, eh) in facess:
                # cropped image of face
                face_roi = roi_color[ey: ey+eh, ex:ex + ew]
    
    final_image = cv2.resize(face_roi, (224, 224))
    final_image = np.expand_dims(final_image, axis = 0)
    
    Predictions = new_model.predict(final_image)
    
    font2 = cv2.FONT_ITALIC
    font_scale = 1.5
    
    if(np.argmax(Predictions) == 0):
        emotion = 'Angry'
        display_text(emotion)
        
    elif(np.argmax(Predictions) == 1):
        emotion = 'Disgust'
        display_text(emotion)
        
    elif(np.argmax(Predictions) == 2):
        emotion = 'Fear'
        display_text(emotion)
        
    elif(np.argmax(Predictions) == 3):
        emotion = 'Happy'
        display_text(emotion)
        
    elif(np.argmax(Predictions) == 4):
        emotion = 'Neutral'
        display_text(emotion)
        
    elif(np.argmax(Predictions) == 5):
        emotion = 'Sad'
        display_text(emotion)
        
    elif(np.argmax(Predictions) == 6):
        emotion = 'Surprised'
        display_text(emotion)
        
    cv2.imshow('Face Emotion Recognition', frame)
    
    if cv2.waitKey(2) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 88ms/step
Face not detected!
1/1 [==============================] - 0s 91ms/step


In [18]:
print(emotion)

Sad


# Converting from text to speech

In [19]:
text_speech = pyttsx3.init()
text = "YOUR DETECTED MOOD IS....." + emotion

# helps us to convert the written text to speech
text_speech.say(text)
# to exit from the program
text_speech.runAndWait()

# Picking the song from the dataset based on the emotion

In [20]:
df = pd.read_excel('mood-based-songs.xlsx', sheet_name = emotion)
df.head()

,Sl No,Name,Artist
0,1,I Took A Pill in Ibiza,Mike Posner
1,2,Human,Rag'n'Bone Man
2,3,Move On,Mike Posner
3,4,Night Changes,One Direction
4,5,Underwater,Mika


### Choosing a row at random from this dataset

In [21]:
random_row = df.sample(n=1)
random_row

,Sl No,Name,Artist
2,3,Move On,Mike Posner


### Converting to list

In [22]:
random_row_list = random_row.values.tolist()
random_row_list

[[3, 'Move On', 'Mike Posner']]

In [23]:
# Concatenating the song name and artist texts from the dataset and storing it in 'song_name'
song_name = random_row_list[0][1] + ' ' + random_row_list[0][2]
song_name

'Move On Mike Posner'

In [ ]:
os.system('spotify')
time.sleep(5)

# activates spotify search bar
pyautogui.hotkey('ctrl', 'l')
time.sleep(3)
# writing in search bar
pyautogui.write(song_name, interval = 0.1)
time.sleep(3)

for key in ['enter', 'pagedown', 'tab', 'enter', 'enter']:
    time.sleep(2)
    pyautogui.press(key)